In [5]:
import pandas as pd
import numpy as np
import seaborn as sns
import nltk
import string

In [6]:
df_train=pd.read_csv('train.csv')
df_test=pd.read_csv('test.csv')
df_train

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


In [7]:
df_train.info()
df_train.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        7613 non-null   int64 
 1   keyword   7552 non-null   object
 2   location  5080 non-null   object
 3   text      7613 non-null   object
 4   target    7613 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 297.5+ KB


id             0
keyword       61
location    2533
text           0
target         0
dtype: int64

In [8]:
df_train['keyword'].value_counts()

fatalities               45
deluge                   42
armageddon               42
sinking                  41
damage                   41
                         ..
forest%20fire            19
epicentre                12
threat                   11
inundation               10
radiation%20emergency     9
Name: keyword, Length: 221, dtype: int64

In [9]:
df_train['location'].value_counts()

USA                    104
New York                71
United States           50
London                  45
Canada                  29
                      ... 
MontrÌ©al, QuÌ©bec       1
Montreal                 1
ÌÏT: 6.4682,3.18287      1
Live4Heed??              1
Lincoln                  1
Name: location, Length: 3341, dtype: int64

In [10]:
df_train.fillna('',inplace=True)

In [11]:
df_train['text_merged'] = df_train['keyword'] + df_train['location'] + df_train['text']

In [12]:
from nltk.corpus import stopwords

In [13]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\vijay\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [14]:
stopwords=nltk.corpus.stopwords.words('english')

In [15]:
def text_process(text):
    
    nopunc=([s for s in text if not s in string.punctuation])
    nopunc="".join (nopunc)
    return [word for word in nopunc.split() if word.lower() not in stopwords]

In [16]:
tfidf=TfidfVectorizer(analyzer=text_process)
X=df_train['text_merged']
y=df_train['target']

In [17]:
X

0       Our Deeds are the Reason of this #earthquake M...
1                  Forest fire near La Ronge Sask. Canada
2       All residents asked to 'shelter in place' are ...
3       13,000 people receive #wildfires evacuation or...
4       Just got sent this photo from Ruby #Alaska as ...
                              ...                        
7608    Two giant cranes holding a bridge collapse int...
7609    @aria_ahrary @TheTawniest The out of control w...
7610    M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...
7611    Police investigating after an e-bike collided ...
7612    The Latest: More Homes Razed by Northern Calif...
Name: text_merged, Length: 7613, dtype: object

In [18]:
from sklearn.model_selection import RandomizedSearchCV,GridSearchCV,train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [30]:
steps_lgbm=[("tfidfVectorizer",TfidfVectorizer(analyzer=text_process)),
      ("lgbmclassifier",LGBMClassifier())]
pipelgbm=Pipeline(steps_lgbm)
pipelgbm.fit(X_train,y_train)
pred=pipelgbm.predict(X_test)
print(classification_report(pred,y_test))
print(confusion_matrix(pred,y_test))
print(accuracy_score(pred,y_test))

              precision    recall  f1-score   support

           0       0.88      0.70      0.78      1650
           1       0.49      0.74      0.59       634

    accuracy                           0.71      2284
   macro avg       0.68      0.72      0.68      2284
weighted avg       0.77      0.71      0.72      2284

[[1154  496]
 [ 164  470]]
0.7110332749562172


In [31]:
steps_mnb=[("tfidfVectorizer",TfidfVectorizer(analyzer=text_process)),
      ("mnbclassifier",MultinomialNB())]
pipemnb=Pipeline(steps_mnb)
pipemnb.fit(X_train,y_train)
pred=pipemnb.predict(X_test)
print(classification_report(pred,y_test))
print(confusion_matrix(pred,y_test))
print(accuracy_score(pred,y_test))

              precision    recall  f1-score   support

           0       0.90      0.77      0.83      1549
           1       0.63      0.82      0.71       735

    accuracy                           0.78      2284
   macro avg       0.76      0.79      0.77      2284
weighted avg       0.81      0.78      0.79      2284

[[1187  362]
 [ 131  604]]
0.7841506129597198


In [32]:
steps_xgb=[("tfidfVectorizer",TfidfVectorizer(analyzer=text_process)),
      ("xgbclassifier",XGBClassifier())]
pipexgb=Pipeline(steps_xgb)
pipexgb.fit(X_train,y_train)
pred=pipexgb.predict(X_test)
print(classification_report(pred,y_test))
print(confusion_matrix(pred,y_test))
print(accuracy_score(pred,y_test))

              precision    recall  f1-score   support

           0       0.92      0.71      0.80      1715
           1       0.48      0.82      0.61       569

    accuracy                           0.74      2284
   macro avg       0.70      0.76      0.70      2284
weighted avg       0.81      0.74      0.75      2284

[[1215  500]
 [ 103  466]]
0.7359894921190894


In [33]:
steps_rf=[("tfidfVectorizer",TfidfVectorizer(analyzer=text_process)),
      ("rfclassifier",RandomForestClassifier())]
piperf=Pipeline(steps_rf)
piperf.fit(X_train,y_train)
pred=piperf.predict(X_test)
print(classification_report(pred,y_test))
print(confusion_matrix(pred,y_test))
print(accuracy_score(pred,y_test))

              precision    recall  f1-score   support

           0       0.89      0.75      0.81      1553
           1       0.60      0.79      0.68       731

    accuracy                           0.76      2284
   macro avg       0.74      0.77      0.75      2284
weighted avg       0.79      0.76      0.77      2284

[[1167  386]
 [ 151  580]]
0.7648861646234676


In [34]:
steps_ab=[("tfidfVectorizer",TfidfVectorizer(analyzer=text_process)),
      ("abclassifier",AdaBoostClassifier())]
pipeab=Pipeline(steps_ab)
pipeab.fit(X_train,y_train)
pred=pipeab.predict(X_test)
print(classification_report(pred,y_test))
print(confusion_matrix(pred,y_test))
print(accuracy_score(pred,y_test))

              precision    recall  f1-score   support

           0       0.94      0.67      0.78      1838
           1       0.38      0.82      0.52       446

    accuracy                           0.70      2284
   macro avg       0.66      0.74      0.65      2284
weighted avg       0.83      0.70      0.73      2284

[[1236  602]
 [  82  364]]
0.7005253940455342


In [35]:
steps_gb=[("tfidfVectorizer",TfidfVectorizer(analyzer=text_process)),
      ("gbclassifier",GradientBoostingClassifier())]
pipegb=Pipeline(steps_gb)
pipegb.fit(X_train,y_train)
pred=pipegb.predict(X_test)
print(classification_report(pred,y_test))
print(confusion_matrix(pred,y_test))
print(accuracy_score(pred,y_test))

              precision    recall  f1-score   support

           0       0.96      0.67      0.79      1875
           1       0.37      0.86      0.51       409

    accuracy                           0.71      2284
   macro avg       0.66      0.77      0.65      2284
weighted avg       0.85      0.71      0.74      2284

[[1262  613]
 [  56  353]]
0.707092819614711


In [37]:
steps_log=[("tfidfVectorizer",TfidfVectorizer(analyzer=text_process)),
      ("logclassifier",LogisticRegression())]
pipelog=Pipeline(steps_log)
pipelog.fit(X_train,y_train)
pred=pipelog.predict(X_test)
print(classification_report(pred,y_test))
print(confusion_matrix(pred,y_test))
print(accuracy_score(pred,y_test))

              precision    recall  f1-score   support

           0       0.91      0.77      0.83      1557
           1       0.62      0.83      0.71       727

    accuracy                           0.79      2284
   macro avg       0.77      0.80      0.77      2284
weighted avg       0.82      0.79      0.79      2284

[[1194  363]
 [ 124  603]]
0.7867775831873906


In [38]:
steps_sdg=[("tfidfVectorizer",TfidfVectorizer(analyzer=text_process)),
      ("sgdclassifier",SGDClassifier())]
pipesdg=Pipeline(steps_sdg)
pipesdg.fit(X_train,y_train)
pred=pipesdg.predict(X_test)
print(classification_report(pred,y_test))
print(confusion_matrix(pred,y_test))
print(accuracy_score(pred,y_test))

              precision    recall  f1-score   support

           0       0.82      0.79      0.81      1362
           1       0.71      0.74      0.73       922

    accuracy                           0.77      2284
   macro avg       0.77      0.77      0.77      2284
weighted avg       0.78      0.77      0.77      2284

[[1082  280]
 [ 236  686]]
0.7740805604203153


In [41]:
log_parameter={'logclassifier__penalty':['l1','l2','elasticnet','none'],
               'logclassifier__dual':[False,True],
               'logclassifier__C':[1,2,3,4,5,6,10,20,30,40,50],
               'logclassifier__max_iter':[100,200,300,400,500,600,700]}
logmodel_hyper=GridSearchCV(pipelog,param_grid=log_parameter,cv=5)
logmodel_hyper.fit(X_train,y_train)
pred=logmodel_hyper.predict(X_test)

C:\Users\vijay\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:1483: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
C:\Users\vijay\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:1483: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
C:\Users\vijay\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:1483: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
C:\Users\vijay\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:1483: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
C:\Users\vijay\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:1483: UserWarning: Setting penalty='none' will ignore the C a

C:\Users\vijay\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:1483: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
C:\Users\vijay\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:1483: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
C:\Users\vijay\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:1483: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
C:\Users\vijay\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:1483: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
C:\Users\vijay\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:1483: UserWarning: Setting penalty='none' will ignore the C a

C:\Users\vijay\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:1483: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
C:\Users\vijay\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:1483: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
C:\Users\vijay\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:1483: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
C:\Users\vijay\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:1483: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
C:\Users\vijay\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:1483: UserWarning: Setting penalty='none' will ignore the C a

C:\Users\vijay\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:1483: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
C:\Users\vijay\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:1483: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
C:\Users\vijay\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:1483: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
C:\Users\vijay\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:1483: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
C:\Users\vijay\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:1483: UserWarning: Setting penalty='none' will ignore the C a

KeyboardInterrupt: 

In [ ]:
print(accuracy_score(pred,y_test))
print(classification_report(pred,y_test))
print(confusion_matrix(pred,y_test))

In [156]:
gbmodel_hyper.get_params().keys()

dict_keys(['cv', 'error_score', 'estimator__memory', 'estimator__steps', 'estimator__verbose', 'estimator__gbclassifier', 'estimator__gbclassifier__ccp_alpha', 'estimator__gbclassifier__criterion', 'estimator__gbclassifier__init', 'estimator__gbclassifier__learning_rate', 'estimator__gbclassifier__loss', 'estimator__gbclassifier__max_depth', 'estimator__gbclassifier__max_features', 'estimator__gbclassifier__max_leaf_nodes', 'estimator__gbclassifier__min_impurity_decrease', 'estimator__gbclassifier__min_samples_leaf', 'estimator__gbclassifier__min_samples_split', 'estimator__gbclassifier__min_weight_fraction_leaf', 'estimator__gbclassifier__n_estimators', 'estimator__gbclassifier__n_iter_no_change', 'estimator__gbclassifier__random_state', 'estimator__gbclassifier__subsample', 'estimator__gbclassifier__tol', 'estimator__gbclassifier__validation_fraction', 'estimator__gbclassifier__verbose', 'estimator__gbclassifier__warm_start', 'estimator', 'n_jobs', 'param_grid', 'pre_dispatch', 'refi

In [ ]:
gb_parameter={'gbclassifier__n_estimators':[80,90,100,110,150],
           'gbclassifier__criterion': ['friedman_mse', 'squared_error', 'mse'],
           'gbclassifier__max_depth': [1,2,3,4,5],
           'gbclassifier__max_features':['auto','sqrt','log2']}

gbmodel_hyper=GridSearchCV(pipegb,param_grid=gb_parameter,scoring='accuracy',cv=5)
gbmodel_hyper.fit(X_train,y_train)
pred=gbmodel_hyper.predict(X_test)

In [229]:
print(accuracy_score(pred,y_test))
print(classification_report(pred,y_test))
print(confusion_matrix(pred,y_test))

0.835820895522388
              precision    recall  f1-score   support

           0       0.94      0.81      0.87       181
           1       0.69      0.89      0.78        87

    accuracy                           0.84       268
   macro avg       0.81      0.85      0.82       268
weighted avg       0.86      0.84      0.84       268

[[147  34]
 [ 10  77]]


In [230]:
gbmodel_hyper.best_params_

{'gbclassifier__criterion': 'squared_error',
 'gbclassifier__max_depth': 4,
 'gbclassifier__max_features': 'log2',
 'gbclassifier__n_estimators': 100}

In [125]:
df_test.fillna('',inplace=True)

df_test['text_merged'] = df_test['keyword'] + df_test['location'] + df_test['text']

In [127]:
pred=pipelog.predict((df_test['text_merged']))
pred

array([1, 1, 1, ..., 1, 1, 1], dtype=int64)

In [128]:
ans=pd.concat([df_test['id'],pd.DataFrame(pred)],axis=1)

In [129]:
ans

,id,0
0,0,1
1,2,1
2,3,1
3,9,1
4,11,1
...,...,...
3258,10861,0
3259,10865,0
3260,10868,1
3261,10874,1


In [130]:
ans.columns=['id','target']

In [131]:
ans.to_csv('submission.csv',index=False)